In [ ]:
"hi"

In [ ]:
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from src.plotting_tools.Bins import bins
import ROOT as r
import array

In [ ]:
from src.plotting_tools.utils import calc_bin_widths, calc_bin_centers
from src.plotting_tools.SysHist import SysHist
from src.data_tools.StackPlotter import get_stack_plotter

In [ ]:
def make_hist(values, errors, bin_edges, *args):
    hpx    = r.TH1F(*args, len(bin_edges)-1, array.array('d', bin_edges))
    for i, (x,e) in enumerate(zip(values,errors)):
        hpx.SetBinContent(i, x) 
        hpx.SetBinError(i, e) 
    return hpx


In [ ]:
# http://cms-analysis.github.io/HiggsAnalysis-CombinedLimit/part2/settinguptheanalysis/
# https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df_input.csv

In [ ]:
mass_hist_template = 'sig{}'

In [ ]:
outdir = 'assets_feb_23'
era = '2016'
rootfname = '{outdir}/combine_data/{era}/{era}_shapes_df_input.root'.format(outdir=outdir, era=era)

In [ ]:
sp = get_stack_plotter(outdir, era, bins='none')

In [ ]:
with open('{}/data/{}_bff_interp.pkl'.format(outdir, era), 'rb') as f:
    data = pkl.load(f)

In [ ]:
outname="{}/abcd/abcd_dict_data_{}_ismc0.pkl".format(outdir, era)
with open(outname,'rb') as f:
    abcd = pkl.load(f)
    
outname="{}/abcd/ABCD_closure_unc.pkl".format(outdir, era)
with open(outname, 'rb') as f:
    uncertainty_dict= pkl.load(f)
uncertainty_dict = uncertainty_dict[int(era)]

In [ ]:
def rebin(oldbins, newbins, data, drawplot=False):
    fig, ax = plt.subplots()
    values = ax.hist(oldbins, newbins, weights = data)
    if not drawplot: plt.close(fig)
    return values[0]

In [ ]:
def rebin_np(oldbins, newbins, data, drawplot=False):
    values = np.histogram(oldbins, newbins, weights = data)
    return values[0]

In [ ]:
from time import perf_counter

In [ ]:

sys_map= {
"{reg}_nom": "nominal", 
"{reg}_jet_jerUp_muon_corrected_pt_ele_pt_{sys_dir}": "jer_{era}_{sys_dir}",
"{reg}_jet_jesTotalUp_muon_corrected_pt_ele_pt_{sys_dir}": "jes_{era}_{sys_dir}",
"{reg}_jet_nom_muon_correctedUp_pt_ele_pt_{sys_dir}": "roch_{era}_{sys_dir}",
"{reg}_jet_jesHEMIssueUp_muon_corrected_pt_ele_pt_{sys_dir}": "HEM_{era}_{sys_dir}",
"{reg}_Weight_BTagCorrUp_{sys_dir}": "btagCorr_{sys_dir}",
"{reg}_Weight_BTagUncorrUp_{sys_dir}": "btagUncorr_{era}_{sys_dir}",
"{reg}_Weight_ElectronSFUp_{sys_dir}": "elSF_{era}_{sys_dir}",
"{reg}_Weight_ISRFSR_Up_{sys_dir}": "ISRFSR_{era}_{sys_dir}",
"{reg}_Weight_L1Up_{sys_dir}": "L1_{era}_{sys_dir}",
"{reg}_Weight_MuonSFUp_{sys_dir}": "Muon_{era}_{sys_dir}",
"{reg}_Weight_MuonTriggerUp_{sys_dir}": "trigger_{era}_{sys_dir}",
"{reg}_Weight_PDF_Up_{sys_dir}": "pdf_{era}_{sys_dir}",
"{reg}_Weight_PUIDUp_{sys_dir}": "puid_{era}_{sys_dir}",
"{reg}_Weight_PuUp_{sys_dir}": "pu_{sys_dir}"}


#no era in PU as correlated

In [ ]:
import re
def get_dbs(string):
    dbs = re.findall('([0-9].[0-9]{2})', string)
    if len(dbs)==1: return float(dbs[0])
    return 0

In [ ]:
dbses  = [0 , 0.04, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]
masses = np.linspace(125,400,int((400-125)/7+1), dtype=int)
#dbses = [0 , 0.04]
#masses = masses[0:1]
masses, dbses

In [ ]:
#hfile = r.TFile('./exo-datacards/EXO-22-006/{era}/{era}_background.root'.format(era=era), 'RECREATE')
#c1 = r.TCanvas( 'c1', 'Dynamic Filling Example', 200, 10, 700, 500 )

In [ ]:
hfile = r.TFile(rootfname, 'RECREATE')
c1 = r.TCanvas( 'c1', 'combine canvas', 200, 10, 700, 500 )

In [ ]:
# abcd prediction
#bin_list = []

for reg in abcd:
    print(reg)
    bin_widths = calc_bin_widths(abcd[reg]['bins'])
    bin_centers = calc_bin_centers(abcd[reg]['bins'])
    noms = abcd[reg]['nom']
    # not statistical error, so scale with width?
    stds = abcd[reg]['std']
    #for i, (nom, std) in enumerate(zip(noms, stds)):
    #    bin_list.append({"channel": reg, 
    #                     "systematic": "sys_0_nominal",
    #                     "bin": i, "sum_w": nom, "sum_ww": std**2,
    #                    "process": 'background'
    #                    })        
    #    bin_list.append({"channel": reg, 
    #                     "systematic": "sys_0_nominal",
    #                     "bin": i, "sum_w": nom, "sum_ww": std**2,
    #                    "process": 'data_obs'
    #                    })  
    hist_name = '{}-{}-{}'.format(reg, "sys_0_nominal", 'data_obs')
    hist = make_hist(noms, stds, abcd[reg]['bins'],  hist_name, hist_name)
    hist.Write()

    print(hist_name)
    hist_name = '{}-{}-{}'.format(reg, "sys_0_nominal", 'background')
    hist =  make_hist(noms, stds, abcd[reg]['bins'], hist_name, hist_name)


#abcd_bin_list = bin_list
#df = pd.DataFrame(bin_list)
#df['channel'] = df.channel.astype('str')
#df['systematic'] = df.systematic.astype('str')
#df['process'] = df.process.astype('str')
#exodc_path = './exo-datacards/EXO-22-006/2017/{}_abcd_df_input.csv'.format(era)
#df[['channel', 'process', 'systematic', 'bin', 'sum_w','sum_ww']].to_csv(exodc_path, index=False)

In [ ]:
plt.errorbar(bin_centers, noms, yerr=stds)

In [ ]:
# why is this off by a factor of e???????
plt.plot(calc_bin_centers(abcd['CR13']['bins']), abcd['CR13']['nom']*abcd['CR10']['nom']/abcd['CR14']['nom'])
plt.plot(bin_centers, abcd['SR1']['nom'])

In [ ]:
def get_normalization(mass, reg, dbs):
    return sp.plot_df[(sp.plot_df.mass==mass) & 
              (sp.plot_df.dbs==dbs) & 
              (sp.plot_df.reg==reg) & 
              (sp.plot_df.feature=='DiLepMass')].iloc[0].nom.sum()

In [ ]:
from scipy.interpolate import LinearNDInterpolator

In [ ]:
interpolators = {}
for reg in ['SR1', 'SR2']:
    mass_arr = []
    dbs_arr = []
    norm_arr = []
    for mass in sp.plot_df.sort_values(['mass']).mass.unique():
        if not mass > 0: continue
        for dbs in sp.plot_df.sort_values(['dbs']).dbs.unique():
            if not dbs > 0: continue
            try:
                norm = get_normalization(mass, reg, dbs)
                print(mass,dbs, norm)
                norm_arr.append(norm)
                mass_arr.append(mass)
                dbs_arr.append(dbs)
            except:
                continue
    X, Y = np.meshgrid(mass_arr, dbs_arr)  # 2D grid for interpolation
    interp = LinearNDInterpolator(list(zip(mass_arr, dbs_arr)), norm_arr)            
    interpolators[reg] = interp

In [ ]:
interpolators['SR1']([202], [0.041])

In [ ]:
bin_edges = abcd[reg]['bins']
for mass in masses:
    #bin_list = []
    for reg in ['SR1', 'SR2']:
        for key in data[mass]:
            norm = interpolators[reg]([mass], [0.04])[0]
            nom = rebin_np(data[mass]['bins'].bin_edges, bin_edges,
                    data[mass]['{}_nom_0.00'.format(reg)])
            for dbs in dbses:
                if key == 'bins': continue
                if reg not in key: continue
                sys_dir = 'down' if 'down' in key else 'up'
                sys_dbs = get_dbs(key)
                if get_dbs(key) != dbs: continue
                if key == 'bins': continue
                if reg not in key: continue
                    
                systematic = 'NA'
                for sys in sys_map:
                    if sys.format(reg=reg, era=era, sys_dir=sys_dir)+"_{:.2f}".format(dbs)==key:
                        systematic = sys_map[sys].format(reg=reg, era=era, sys_dir=sys_dir.capitalize())
                        break
                if systematic=='NA': 
                    print("----")
                    for sys in sys_map:
                        print("\t",sys.format(reg=reg, era=era, sys_dir=sys_dir)+"_{:.2f}".format(dbs))
                values = rebin_np(data[mass]['bins'].bin_edges, bin_edges, data[mass][key])

                #print(key, np.sum(values))
                #for i, value in enumerate(values):
                #    if 'Trigger' in key:
                #        value = value +1
                #    bin_list.append({"channel": reg, 
                #                     "systematic": "sys_{}_{}".format(dbs, systematic),
                #                     "bin": i, "sum_w": value, "sum_ww": value,
                #                    "process": int(mass)
                #                    })
                mass_proc = mass_hist_template.format(int(mass))
                hist_name = '{}-{}-{}'.format(reg, "sys_{}_{}".format(dbs, systematic),  mass_proc)
                hist = make_hist(values*norm, values*.01*norm, bin_edges, hist_name, hist_name)
                hist.Write()
            
        #bin_list += abcd_bin_list
        #make df
        #df = pd.DataFrame(bin_list)
        #df['channel'] = df.channel.astype('str')
        #df['systematic'] = df.systematic.astype('str')
        #df['process'] = df.process.astype('str')        
        #exodc_path = '/afs/cern.ch/work/r/rymuelle/public/nanoAODzPrime/CMSSW_12_1_0/src/bff_plotting_code_v3/exo-datacards/EXO-22-006/2017/{}_{}_shapes_df_input.csv'.format(era, mass)
        #df[['channel', 'process', 'systematic', 'bin', 'sum_w','sum_ww']].to_csv(exodc_path, index=False)
            

In [ ]:
y3_masses = []
for i, row in sp.plot_df.iterrows():
    if 'Y3' != row.category: continue
    if 'DiLepMass' != row.feature: continue
    if not 'SR2' in row.reg: continue
    print(i)
    mass = re.findall('y3_([0-9]+)_', row.file)[0]
    y3_masses.append(mass)
    reg = row.reg
    y3nom = rebin_np(calc_bin_centers(row.bins), bin_edges, row.nom)
    y3std = rebin_np(calc_bin_centers(row.bins), bin_edges, row['std'])
    y3nom_norm = y3nom/np.sum(y3nom)
    systematic='nom'
    hist_name = '{}-{}-{}'.format(reg, "sys_{}_{}".format('Y3', systematic),  'sig'+mass)
    hist = make_hist(y3nom_norm, y3nom_norm*.01, bin_edges, hist_name, hist_name)
    hist.Write()
    print(hist_name)
    print("-----")
    sys_row = row['sys']
    sys_row.keys()
    for sys in sys_row.keys():
        print("name", sys)
        up, down = sys_row[sys]
        sys = sys.replace('SR1','').replace('SR2','')
        for values, sys_dir in zip([up,down], ['up', 'down']):
            _values = values + row.nom
            sign = abs(np.sum(values))/np.sum(values)
            if sys == 'Weight_ISRFSR_Up': 
                _values = sign*.16*row.nom + row.nom
            for _sys in sys_map:
                if sys in _sys: break   
            if not sys in _sys: ""
            _values = _values/np.sum(y3nom)       
            value_rebin = rebin_np(calc_bin_centers(row.bins), bin_edges, _values)
            sys_name = sys_map[_sys].format(reg=reg, era=era, sys_dir=sys_dir.capitalize())
            hist_name = '{}-{}-{}'.format(reg, "sys_{}_{}".format('Y3', sys_name),  'sig'+mass)
            hist = make_hist(value_rebin, value_rebin*.01, bin_edges, hist_name, hist_name)
            hist.Write()
            #print(hist_name)

In [ ]:
# test alignment of abcd and stuff
nom = rebin_np(data[202]['bins'].bin_edges, abcd[reg]['bins'],
                    data[202]['{}_nom_0.00'.format(reg)])
plt.plot(noms)
plt.plot(nom*100)



In [ ]:
def draw_mass(ax, mass):
    _bins = abcd[reg]['bins']
    nom = rebin_np(data[mass]['bins'].bin_edges, _bins,
                    data[mass]['{}_nom_0.00'.format(reg)])
    ax.plot(calc_bin_centers(_bins), nom, label=mass)
    

In [ ]:
data.keys()

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
draw_mass(ax, 188)
draw_mass(ax, 195)
draw_mass(ax, 202)
draw_mass(ax, 209)
draw_mass(ax, 400)
plt.plot(bin_centers, noms/1000)

In [ ]:
#test sys
plt.plot(data[202]['SR1_Weight_PDF_Up_up_1.00']/data[202]['SR1_Weight_PDF_Up_up_0.00'])
plt.plot(data[202]['SR1_Weight_PDF_Up_up_0.50']/data[202]['SR1_Weight_PDF_Up_up_0.00'])
plt.plot(data[202]['SR1_Weight_PDF_Up_up_0.00']/data[202]['SR1_Weight_PDF_Up_up_0.00'])


In [ ]:
if era=="2018":
    template = '''#higgs combine tool shape analysis card for z'to mumu 1 jet
#https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df.txt
-------------------------

imax 1  number of channels                                      #1 Jet
jmax 1  number of backgrounds -1                                    #following AN2015_207_v5, not sure why the -1 is there?
kmax *  number of nuisance parameters (sources of systematic uncertainties)

-------------------------


shapes * * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes data_obs * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes background * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-background

-------------------------

bin       {reg}
observation   -1

-------------------------

bin       {reg}       {reg}
process     background    {mass_proc}
process     1     -1
rate      -1   -1

-------------------------
lumi lnN -      {lumi}
back_closure_{era} lnN {back_closure}    -    
sys_{dbs}_jer_{era}_   shapeN2 -      1
sys_{dbs}_jes_{era}_   shapeN2 -      1
sys_{dbs}_roch_{era}_   shapeN2 -      1
sys_{dbs}_HEM_{era}_   shapeN2 -      1
sys_{dbs}_btagCorr_   shapeN2 -      1
sys_{dbs}_btagUncorr_{era}_   shapeN2 -      1
sys_{dbs}_elSF_{era}_   shapeN2 -      1
sys_{dbs}_ISRFSR_{era}_   shapeN2 -      1
sys_{dbs}_Muon_{era}_   shapeN2 -      1
sys_{dbs}_trigger_{era}_   shapeN2 -      1
sys_{dbs}_pdf_{era}_   shapeN2 -      1
sys_{dbs}_puid_{era}_   shapeN2 -      1
sys_{dbs}_pu_   shapeN2 -      1
--------------------------------------------------------------------------------'''
else:
    template = '''#higgs combine tool shape analysis card for z'to mumu 1 jet
#https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df.txt
-------------------------

imax 1  number of channels                                      #1 Jet
jmax 1  number of backgrounds -1                                    #following AN2015_207_v5, not sure why the -1 is there?
kmax *  number of nuisance parameters (sources of systematic uncertainties)

-------------------------


shapes * * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes data_obs * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes background * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-background

-------------------------

bin       {reg}
observation   -1

-------------------------

bin       {reg}       {reg}
process     background    {mass_proc}
process     1     -1
rate      -1   -1

-------------------------
lumi lnN -      {lumi}
back_closure_{era} lnN {back_closure}    -      
sys_{dbs}_jer_{era}_   shapeN2 -      1
sys_{dbs}_jes_{era}_   shapeN2 -      1
sys_{dbs}_roch_{era}_   shapeN2 -      1
sys_{dbs}_btagCorr_   shapeN2 -      1
sys_{dbs}_btagUncorr_{era}_   shapeN2 -      1
sys_{dbs}_elSF_{era}_   shapeN2 -      1
sys_{dbs}_ISRFSR_{era}_   shapeN2 -      1
sys_{dbs}_L1_{era}_   shapeN2 -      1
sys_{dbs}_Muon_{era}_   shapeN2 -      1
sys_{dbs}_trigger_{era}_   shapeN2 -      1
sys_{dbs}_pdf_{era}_   shapeN2 -      1
sys_{dbs}_puid_{era}_   shapeN2 -      1
sys_{dbs}_pu_   shapeN2 -      1
--------------------------------------------------------------------------------'''        

In [ ]:
if era=='2016':
    lumi=1.025
if era=='2017':
    lumi=1.023
if era=='2018':
    lumi=1.025

In [ ]:
for reg in ['SR1', 'SR2']:
    for mass in masses:
            for dbs in dbses:
                mass_proc = mass_hist_template.format(int(mass))
                tmp = template.format(era=era,dbs=dbs,mass_proc=mass_proc,lumi=lumi, reg=reg,
                                     back_closure = uncertainty_dict[reg])
                #with open('exo-datacards/EXO-22-006/comb_{}_{}_{}.txt'.format(reg, mass, dbs), 'w') as f:
                with open('{}/combine_data/{}/{}_{}_BFFZprimeToMuMu_fit_M_{}_dbs{}.txt'.format(outdir, 
                                                                                               era, era, reg, mass, 
                                                                                                    str(dbs).replace('.','p')), 'w') as f:
                    f.write(tmp)
                

In [ ]:
if era=="2018":
    template = '''#higgs combine tool shape analysis card for z'to mumu 1 jet
#https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df.txt
-------------------------

imax 1  number of channels                                      #1 Jet
jmax 1  number of backgrounds -1                                    #following AN2015_207_v5, not sure why the -1 is there?
kmax *  number of nuisance parameters (sources of systematic uncertainties)

-------------------------


shapes * * {era}_shapes_df_input.root $CHANNEL-sys_Y3_nom-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes data_obs * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes background * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-background

-------------------------

bin       {reg}
observation   -1

-------------------------

bin       {reg}       {reg}
process     background    {mass_proc}
process     1     -1
rate      -1   -1

-------------------------
lumi lnN -      {lumi}
back_closure_{era} lnN {back_closure}    - 
sys_{dbs}_jer_{era}_   shapeN2 -      1
sys_{dbs}_jes_{era}_   shapeN2 -      1
sys_{dbs}_roch_{era}_   shapeN2 -      1
sys_{dbs}_HEM_{era}_   shapeN2 -      1
sys_{dbs}_btagCorr_   shapeN2 -      1
sys_{dbs}_btagUncorr_{era}_   shapeN2 -      1
sys_{dbs}_elSF_{era}_   shapeN2 -      1
sys_{dbs}_ISRFSR_{era}_   shapeN2 -      1
sys_{dbs}_Muon_{era}_   shapeN2 -      1
sys_{dbs}_trigger_{era}_   shapeN2 -      1
sys_{dbs}_pdf_{era}_   shapeN2 -      1
sys_{dbs}_puid_{era}_   shapeN2 -      1
sys_{dbs}_pu_   shapeN2 -      1
--------------------------------------------------------------------------------'''
else:
    template = '''#higgs combine tool shape analysis card for z'to mumu 1 jet
#https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df.txt
-------------------------

imax 1  number of channels                                      #1 Jet
jmax 1  number of backgrounds -1                                    #following AN2015_207_v5, not sure why the -1 is there?
kmax *  number of nuisance parameters (sources of systematic uncertainties)

-------------------------


shapes * * {era}_shapes_df_input.root $CHANNEL-sys_Y3_nom-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes data_obs * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes background * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-background

-------------------------

bin       {reg}
observation   -1

-------------------------

bin       {reg}       {reg}
process     background    {mass_proc}
process     1     -1
rate      -1   -1

-------------------------
lumi lnN -      {lumi}
back_closure_{era} lnN {back_closure}    - 
sys_{dbs}_jer_{era}_   shapeN2 -      1
sys_{dbs}_jes_{era}_   shapeN2 -      1
sys_{dbs}_roch_{era}_   shapeN2 -      1
sys_{dbs}_btagCorr_   shapeN2 -      1
sys_{dbs}_btagUncorr_{era}_   shapeN2 -      1
sys_{dbs}_elSF_{era}_   shapeN2 -      1
sys_{dbs}_ISRFSR_{era}_   shapeN2 -      1
sys_{dbs}_L1_{era}_   shapeN2 -      1
sys_{dbs}_Muon_{era}_   shapeN2 -      1
sys_{dbs}_trigger_{era}_   shapeN2 -      1
sys_{dbs}_pdf_{era}_   shapeN2 -      1
sys_{dbs}_puid_{era}_   shapeN2 -      1
sys_{dbs}_pu_   shapeN2 -      1
--------------------------------------------------------------------------------'''        

In [ ]:
for reg in ['SR1', 'SR2']:
    for mass in y3_masses:
            for dbs in ['Y3']:
                mass_proc = mass_hist_template.format(int(mass))
                tmp = template.format(era=era,dbs=dbs,mass_proc=mass_proc,lumi=lumi, reg=reg,
                                     back_closure = uncertainty_dict[reg])
                #with open('exo-datacards/EXO-22-006/comb_{}_{}_{}.txt'.format(reg, mass, dbs), 'w') as f:
                with open('{}/combine_data/{}/{}_{}_BFFZprimeToMuMu_fit_M_{}_dbs{}.txt'.format(outdir, 
                                                                                               era, era, reg, mass, 
                                                                                                    str(dbs).replace('.','p')), 'w') as f:
                    f.write(tmp)
                

In [ ]:
hfile.Write()

In [ ]:
import uproot as upr
import matplotlib.pyplot as plt
upf = upr.open(rootfname)

In [ ]:
[x for x in upf.keys() if '202' in x]

In [ ]:
[x for x in upf.keys() if 'ack' in x]

In [ ]:
upf['SR1-sys_0_nominal-sig202'].values().sum(), upf['SR2-sys_0_nominal-sig202'].values().sum()

In [ ]:
upf['SR1-sys_1.0_jer_2016_Down-sig202'].values().sum(), upf['SR1-sys_1.0_jer_2016_Up-sig202'].values().sum()

In [ ]:
bin_centers = np.array(bin_centers)

In [ ]:
window = (bin_centers > 200-20) & (bin_centers < 200+20)
window

In [ ]:
plt.plot(bin_centers[window], upf['SR1-sys_0_nominal-background'].values()[window])

In [ ]:
upf['SR1-sys_0_nominal-background'].values()[window].sum(), upf['SR1-sys_0_nominal-sig202'].values()[window].sum()

In [ ]:
from src.data_tools.StackPlotter import get_stack_plotter

In [ ]:
sp16 = get_stack_plotter(outdir, '2016', bins='none')

In [ ]:
from src.assets.lumi import lumi_dict

In [ ]:
hist = sp16.combine_back('DiLepMass', 'SR1')
hist.reduce_range(220, 180).calc_sum()*lumi_dict['201X']/lumi_dict['2016']

In [ ]:
upf['SR1-sys_0_nominal-background'].values()[window].sum()*lumi_dict['201X']/lumi_dict['2016']

In [ ]:
# produce plots with these params
mass = 251
dbs = 0.4

In [ ]:
re_patern = 'SR1-sys_{}.+Up-sig{}'.format(dbs, mass)
[x for x in upf.keys() 
           if len(re.findall(re_patern,x)) > 0]

In [ ]:
# create lists of systematics
re_patern = 'SR1-sys_{}.+Up-sig{}'.format(dbs, mass)
syskeys = [x for x in upf.keys() 
           if len(re.findall(re_patern,x)) > 0]
syskeys = [x.replace('Up', '{}')for x in syskeys]
# also define nominal key
nomkey = 'SR1-sys_0_nominal-sig{}'.format(mass)

syskeys

In [ ]:
def calc_center(bins):
    return np.array([(bins[i]+bins[i+1])/2 for i in range(len(bins)-1)])

def plot_from_key(ax, key):
    xbins = upf[key].tojson()['fXaxis']['fXbins']
    xbin_center = calc_center(xbins)
    var = upf[key].variances()
    values = upf[key].values()
    mean = sum(values*xbin_center)/values.sum()
    std = sum((values*(xbin_center-mean)**2)/values.sum() )*.5
    print(mean, np.sum(values))
    return ax.step(xbin_center, values, where='mid'), mean, std

In [ ]:
fig, axs = plt.subplots(2,1)
_, mean, std = plot_from_key(axs[0], nomkey)
_, mean, std = plot_from_key(axs[0], nomkey.replace('SR1','SR2'))
_, mean, std = plot_from_key(axs[0], 'SR1-sys_0_nominal-background')

_, mean, std = plot_from_key(axs[0], 'SR2-sys_Y3_nom-sig250')
_, mean, std = plot_from_key(axs[0], 'SR2-sys_Y3_jes_2016_Up-sig250')
_, mean, std = plot_from_key(axs[0], 'SR2-sys_Y3_jes_2016_Down-sig250')

axs[0].set_yscale('log')
#axs[0].set_xlim(mean-std,mean+std)
#axs[1].set_xlim(mean-std,mean+std)

In [ ]:
# test for density plot
nomkey = 'SR1-sys_0_nominal-sig{}'.format(202)
bff = upf[nomkey].values()
print("bff")
abcd = upf['SR1-sys_0_nominal-background'].values()

In [ ]:
bw = bin_widths.copy()

In [ ]:
bw.append(5.0)

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
ax.set_xlim(190,210)
plt.plot(bin_centers,bin_widths, marker='o')
plt.plot(bin_centers,bw[1:], marker='o')

In [ ]:
plt.plot(bin_centers, abcd/bin_widths, marker='o')
plt.plot(bin_centers, abcd/bw[1:], marker='o')



In [ ]:
fig, ax = plt.subplots(figsize=(12,10))

plt.plot(bin_centers, bff/bw[1:], marker='o', label='bff adj')
plt.plot(bin_centers, bff/bin_widths, marker='o', label='bff adj')

plt.plot(bin_centers, abcd/bw[1:]*1e-3, marker='o', label='abcd adj')
plt.plot(bin_centers, abcd/bin_widths*1e-3, marker='o', label='abcd')
ax.set_xlim(190,210)
plt.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))

plt.plot(bin_centers, bff, marker='o')
plt.plot(bin_centers, abcd*1e-3, marker='o')
ax.set_xlim(190,210)